In [23]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "7"   #(xxxx is your specific GPU ID)

In [24]:
import numpy as np
import torch
import torch.nn as nn
import math
import torchvision.models as models
from torch.utils.data import DataLoader, TensorDataset
from torchsummary import summary
import copy
import matplotlib.pyplot as plt
import pandas as pd
import timeit
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import scipy.io
import scaleogram as scg
import shutil
from PIL import Image
from torchvision import transforms
import glob

In [25]:
if torch.cuda.is_available():
    dev = "cuda:0" 
else:
    dev = "cpu" 
device = torch.device(dev) 
device

device(type='cuda', index=0)

In [26]:
print(torch.__version__)

1.10.1+cu102


# sCDP

In [27]:
command_dict = {'Backward': 0,
 'Birds': 1,
 'Continue': 2,
 'Down': 3,
 'Forward': 4,
 'Four': 5,
 'Landing': 6,
 'Left': 7,
 'Light': 8,
 'Look': 9,
 'Off': 10,
 'On': 11,
 'One': 12,
 'Reverse': 13,
 'Right': 14,
 'Rotate': 15,
 'Side': 16,
 'Snapshot': 17,
 'Stop': 18,
 'Task': 19,
 'Three': 20,
 'Two': 21,
 'Up': 22}
command_dict = {v: k for k, v in command_dict.items()}
command_dict

{0: 'Backward',
 1: 'Birds',
 2: 'Continue',
 3: 'Down',
 4: 'Forward',
 5: 'Four',
 6: 'Landing',
 7: 'Left',
 8: 'Light',
 9: 'Look',
 10: 'Off',
 11: 'On',
 12: 'One',
 13: 'Reverse',
 14: 'Right',
 15: 'Rotate',
 16: 'Side',
 17: 'Snapshot',
 18: 'Stop',
 19: 'Task',
 20: 'Three',
 21: 'Two',
 22: 'Up'}

In [28]:
###################### model 2 (keyword)###################### 

class keyword_net(nn.Module):
    def __init__(self):
        super(keyword_net, self).__init__()
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=1)            
        self.cvd = nn.Conv2d(in_channels=7, out_channels=3, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1)) 
        self.fc1_1 = nn.Linear(71910,128) 
        self.fc1_2 = nn.Linear(128,256) 
        self.keyword_output_1 = nn.Linear(256,11)
        self.fc2_1 = nn.Linear(71910,512) # Dense_2_1 = tf.keras.layers.Dense(shape_2_1, activation=actv_fun_2_1,name='fc2_1')
        self.fc2_2 = nn.Linear(512,512) # tf.keras.layers.Dense(shape_2_2, activation=actv_fun_2_2,name='fc2_2')
        self.getalphafromy2 = nn.Linear(512,512)
        self.fc2_3 = nn.Linear(1024,512)
        self.keyword_output_2 = nn.Linear(512,23) # tf.keras.layers.Dense(10, activation='softmax',name='command_output')


    def forward(self, x):
   
        # convert 512*7*7 to 512*49
#         print("before:" , x.shape)
        x =  x.reshape(x.size(0), x.size(1), x.size(2), -1)
#         print("after:" , x.shape)
        x = self.cvd(x)
        x = x.view(x.size(0), -1) 
        
        y1 = self.fc1_1(x)
        y1 = self.relu(y1)
        y1 = self.fc1_2(y1)
        y1 = self.sigmoid(y1) 
        y1 = self.keyword_output_1(y1)
        y1 = self.softmax(y1)
        
        y2 = self.fc2_1(x)
        y2 = self.relu(y2)
        y2 = self.fc2_2(y2)
        y2 = self.sigmoid(y2) 
        y2 = self.keyword_output_2(y2)
        y2 = self.softmax(y2)
        
        return y1,y2

In [29]:
class CombinedModel(nn.Module):
    def __init__(self, PRETRAIN_MODEL,keyword_net):
        super(CombinedModel, self).__init__()
        self.pretrain_model = PRETRAIN_MODEL
        self.keyword_net = keyword_net
        
    def forward(self, x):
#         print(x.shape)
#         print(x[:][0].shape)
        x1 = self.pretrain_model(x[:,0,:,:])
        x1 = torch.unsqueeze(x1, dim=1)
#         print(x1.shape)
        x2 = self.pretrain_model(x[:,1,:,:])
        x2 = torch.unsqueeze(x2, dim=1)
        x3 = self.pretrain_model(x[:,2,:,:])
        x3 = torch.unsqueeze(x3, dim=1)
        x4 = self.pretrain_model(x[:,3,:,:])
        x4 = torch.unsqueeze(x4, dim=1)        
        x5 = self.pretrain_model(x[:,4,:,:])
        x5 = torch.unsqueeze(x5, dim=1)        
        x6 = self.pretrain_model(x[:,5,:,:])
        x6 = torch.unsqueeze(x6, dim=1)        
        x7 = self.pretrain_model(x[:,6,:,:]) 
        x7 = torch.unsqueeze(x7, dim=1)      
                
        x = torch.cat((x1, x2, x3, x4, x5, x6, x7), axis=1)
#         print(x.shape)
        y1,y2= self.keyword_net(x)
        return y1,y2
        

In [ ]:
PATH = "Final_model/sCPD_group_Featurefuse_unfreeze_0412"
resnet = models.resnet18(pretrained=True)
PRETRAIN_MODEL = torch.nn.Sequential(*(list(resnet.children())[:-2])) 
model = CombinedModel(PRETRAIN_MODEL,keyword_net())
model.load_state_dict(torch.load(PATH))
model.to(device)
summary(model, ( 7,3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [31]:
# alignment file for later use 
start = timeit.default_timer()
keywords = ['Backward','Birds','Continue','Down','Forward', 'Four','Landing','Left','Light','Look','Off','On','One',
    'Reverse','Right','Rotate','Side','Snapshot','Stop','Task','Three','Two','Up']
sr = 40000
subject_number = 11
Alignment = pd.read_csv("Alignment_label_final.csv")
# remove the mannully revised or removed file
removed_alignment = Alignment[(Alignment['Manually_Revised']==True)|(Alignment['Manually_Removed']==True)]
removed_alignment = removed_alignment.reset_index()
new_Alignment =  Alignment[(Alignment['Manually_Removed']==False)]
new_Alignment = new_Alignment.reset_index(drop=True)

stop = timeit.default_timer()
run_time = stop - start
run_time

0.03248332999646664

In [ ]:
s = 5 # s is the subject No., here from 0 to 10, set a No. to get this subject data, e.g., set s=0 to get subject 1 data
sub = str(s+1).zfill(2) # get the subject #
word = 'Continue' # word is the keyword, select one from the list of keywords

if os.path.exists('inference_time_images'):
    shutil.rmtree('inference_time_images')
output_dir = "inference_time_images/test"
os.makedirs(output_dir, exist_ok=True)
start = timeit.default_timer()

#################################### convet the original mat file to scalogram image ####################################

################### get signal data from mat file 

#get the mat file of this word from the subject s and get the labels of all segmentations
mat_path = 'raw_data/subject_'+sub+'/'+word+'.mat'
mat = scipy.io.loadmat(mat_path)
labels = new_Alignment[(new_Alignment ["keyword"] == word)&(new_Alignment ["subject"] == s+1)]

signal_list = []
for Channel in range(1,8): # there are 7 channels for sEMG data in this dataset, from 1,2,...7
    c =  Channel - 1

    # get the data of this channel in this word from the subject s 
    if len(mat['tickrate'][0])>1:# just in case some files have a pause during recording
        for i in range(len(mat['tickrate'][0])): 
            s_data = mat['data'][0][int(mat['datastart'][c][i]):int(mat['dataend'][c][i])]
            s_data = np.transpose(s_data)
            data = np.append(data,s_data) if i > 0 else s_data
    else:
        data = mat['data'][0][int(mat['datastart'][c]):int(mat['dataend'][c])]
        data = np.transpose(data)  


    i = 15 # the index of the segmentation, i in range(len(labels)), in this dataset, i is from 0 to 50+
    label = labels["Label"].values[i]  # to get the lable of the segment i, Train, Val, or Test

    idnum = labels["wav_index"].values[i]-1  # the id of the output figure. 

    signal = data[labels["Start"].values[i]:labels["End"].values[i]] # get the original signal data based on the start and end index
    signal_list.append(signal)

stop = timeit.default_timer()
run_time = stop - start
print(run_time)
# signal_list
# 

################### convert signal data to Scalogram
scales = np.arange(5, 500) 
for Channel in range(1,8): # there are 7 channels for sEMG data in this dataset, from 1,2,...7
    c =  Channel - 1
    signal = signal_list[c]
    
    
#     downfreq_start = timeit.default_timer()
    low_data = []
    for j in range(len(signal)//40):
        low_data.append(signal[j*40:(j+1)*40].mean())
    low_data = np.array(low_data)

#     downfreq_stop = timeit.default_timer()
#     downfreq_run_time = downfreq_stop - downfreq_start
#     print("Channel %d downfreq run time is "%(Channel), downfreq_run_time )

    time = np.linspace(0, 1.5, num=len(low_data)) # 1.5 second per segment
#     print("sEMG Scaleogram of Index %d for subject #%s, keyword of %s, on Channel %d, the lable is %s."%(i,sub,word,Channel,label))

#     scg_start = timeit.default_timer()
    ax = scg.cws(time, low_data, scales, yaxis = 'frequency', yscale='log')
    filename = f"sub_{sub}_idx_{i}_word_{word}_label_{label}_ch_{Channel}.png"
    filepath = os.path.join(output_dir, filename)
    plt.savefig(filepath, bbox_inches='tight',  pad_inches=0) 
    plt.close() 
#     scg_stop = timeit.default_timer()
#     scg_run_time = scg_stop - scg_start
#     print("Channel %d generate scalogram run time is "%(Channel), scg_run_time )
stop = timeit.default_timer()
run_time = stop - start
print(run_time)

################### achrive image and save as an array

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

image_paths = sorted(glob.glob(os.path.join(output_dir, "*.png")))
images = [transform(Image.open(p).convert("RGB")) for p in image_paths]
inputs_tensor = torch.stack(images).view(-1, 7, 3, 224, 224).to(device)

stop = timeit.default_timer()
run_time = stop - start
print(run_time)


################### test the data
sub_res = np.argmax(model(inputs_tensor)[0].cpu()[0].detach().numpy())
comd_res = np.argmax(model(inputs_tensor)[1].cpu()[0].detach().numpy())

print('Result: subject is #%d, and command is %s'%(sub_res+1,command_dict[comd_res]))
stop = timeit.default_timer()
run_time = stop - start
print(run_time)


0.20847055898047984
9.450557173928246
9.485177159891464
Result: subject is #6, and command is Continue
9.541157932952046


In [ ]:
# if change the scale to 1,100
s = 5 # s is the subject No., here from 0 to 10, set a No. to get this subject data, e.g., set s=0 to get subject 1 data
sub = str(s+1).zfill(2) # get the subject #
word = 'Continue' # word is the keyword, select one from the list of keywords

if os.path.exists('inference_time_images'):
    shutil.rmtree('inference_time_images')
output_dir = "inference_time_images/test"
os.makedirs(output_dir, exist_ok=True)
start = timeit.default_timer()

#################################### convet the original mat file to scalogram image ####################################

################### get signal data from mat file 

#get the mat file of this word from the subject s and get the labels of all segmentations
mat_path = 'raw_data/subject_'+sub+'/'+word+'.mat'
mat = scipy.io.loadmat(mat_path)
labels = new_Alignment[(new_Alignment ["keyword"] == word)&(new_Alignment ["subject"] == s+1)]

signal_list = []
for Channel in range(1,8): # there are 7 channels for sEMG data in this dataset, from 1,2,...7
    c =  Channel - 1

    # get the data of this channel in this word from the subject s 
    if len(mat['tickrate'][0])>1:# just in case some files have a pause during recording
        for i in range(len(mat['tickrate'][0])): 
            s_data = mat['data'][0][int(mat['datastart'][c][i]):int(mat['dataend'][c][i])]
            s_data = np.transpose(s_data)
            data = np.append(data,s_data) if i > 0 else s_data
    else:
        data = mat['data'][0][int(mat['datastart'][c]):int(mat['dataend'][c])]
        data = np.transpose(data)  


    i = 15 # the index of the segmentation, i in range(len(labels)), in this dataset, i is from 0 to 50+
    label = labels["Label"].values[i]  # to get the lable of the segment i, Train, Val, or Test

    idnum = labels["wav_index"].values[i]-1  # the id of the output figure. 

    signal = data[labels["Start"].values[i]:labels["End"].values[i]] # get the original signal data based on the start and end index
    signal_list.append(signal)

stop = timeit.default_timer()
run_time = stop - start
print(run_time)
# signal_list
# 

################### convert signal data to Scalogram
scales = np.arange(1, 100) 
for Channel in range(1,8): # there are 7 channels for sEMG data in this dataset, from 1,2,...7
    c =  Channel - 1
    signal = signal_list[c]
    
    
#     downfreq_start = timeit.default_timer()
    low_data = []
    for j in range(len(signal)//40):
        low_data.append(signal[j*40:(j+1)*40].mean())
    low_data = np.array(low_data)

#     downfreq_stop = timeit.default_timer()
#     downfreq_run_time = downfreq_stop - downfreq_start
#     print("Channel %d downfreq run time is "%(Channel), downfreq_run_time )

    time = np.linspace(0, 1.5, num=len(low_data)) # 1.5 second per segment
#     print("sEMG Scaleogram of Index %d for subject #%s, keyword of %s, on Channel %d, the lable is %s."%(i,sub,word,Channel,label))

#     scg_start = timeit.default_timer()
    ax = scg.cws(time, low_data, scales, yaxis = 'frequency', yscale='log')
    filename = f"sub_{sub}_idx_{i}_word_{word}_label_{label}_ch_{Channel}.png"
    filepath = os.path.join(output_dir, filename)
    plt.savefig(filepath, bbox_inches='tight',  pad_inches=0) 
    plt.close() 
#     scg_stop = timeit.default_timer()
#     scg_run_time = scg_stop - scg_start
#     print("Channel %d generate scalogram run time is "%(Channel), scg_run_time )
stop = timeit.default_timer()
run_time = stop - start
print(run_time)

################### achrive image and save as an array

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

image_paths = sorted(glob.glob(os.path.join(output_dir, "*.png")))
images = [transform(Image.open(p).convert("RGB")) for p in image_paths]
inputs_tensor = torch.stack(images).view(-1, 7, 3, 224, 224).to(device)

stop = timeit.default_timer()
run_time = stop - start
print(run_time)


################### test the data
sub_res = np.argmax(model(inputs_tensor)[0].cpu()[0].detach().numpy())
comd_res = np.argmax(model(inputs_tensor)[1].cpu()[0].detach().numpy())

print('Result: subject is #%d, and command is %s'%(sub_res+1,command_dict[comd_res]))
stop = timeit.default_timer()
run_time = stop - start
print(run_time)


0.19358758605085313
2.2696414979873225
2.2989466820145026
Result: subject is #11, and command is Continue
2.3417483390076086


In [ ]:
# if downsampling to about 500 data point per segment

s = 5 # s is the subject No., here from 0 to 10, set a No. to get this subject data, e.g., set s=0 to get subject 1 data
sub = str(s+1).zfill(2) # get the subject #
word = 'Continue' # word is the keyword, select one from the list of keywords

if os.path.exists('inference_time_images'):
    shutil.rmtree('inference_time_images')
output_dir = "inference_time_images/test"
os.makedirs(output_dir, exist_ok=True)
start = timeit.default_timer()

#################################### convet the original mat file to scalogram image ####################################

################### get signal data from mat file 

#get the mat file of this word from the subject s and get the labels of all segmentations
mat_path = 'raw_data/subject_'+sub+'/'+word+'.mat'
mat = scipy.io.loadmat(mat_path)
labels = new_Alignment[(new_Alignment ["keyword"] == word)&(new_Alignment ["subject"] == s+1)]

signal_list = []
for Channel in range(1,8): # there are 7 channels for sEMG data in this dataset, from 1,2,...7
    c =  Channel - 1

    # get the data of this channel in this word from the subject s 
    if len(mat['tickrate'][0])>1:# just in case some files have a pause during recording
        for i in range(len(mat['tickrate'][0])): 
            s_data = mat['data'][0][int(mat['datastart'][c][i]):int(mat['dataend'][c][i])]
            s_data = np.transpose(s_data)
            data = np.append(data,s_data) if i > 0 else s_data
    else:
        data = mat['data'][0][int(mat['datastart'][c]):int(mat['dataend'][c])]
        data = np.transpose(data)  


    i = 15 # the index of the segmentation, i in range(len(labels)), in this dataset, i is from 0 to 50+
    label = labels["Label"].values[i]  # to get the lable of the segment i, Train, Val, or Test

    idnum = labels["wav_index"].values[i]-1  # the id of the output figure. 

    signal = data[labels["Start"].values[i]:labels["End"].values[i]] # get the original signal data based on the start and end index
    signal_list.append(signal)

stop = timeit.default_timer()
run_time = stop - start
print(run_time)
# signal_list
# 

################### convert signal data to Scalogram
scales = np.arange(5, 500) 
for Channel in range(1,8): # there are 7 channels for sEMG data in this dataset, from 1,2,...7
    c =  Channel - 1
    signal = signal_list[c]
    
    
#     downfreq_start = timeit.default_timer()
    low_data = []
    for j in range(len(signal)//120):
        low_data.append(signal[j*120:(j+1)*120].mean())
    low_data = np.array(low_data)

#     downfreq_stop = timeit.default_timer()
#     downfreq_run_time = downfreq_stop - downfreq_start
#     print("Channel %d downfreq run time is "%(Channel), downfreq_run_time )

    time = np.linspace(0, 1.5, num=len(low_data)) # 1.5 second per segment
#     print("sEMG Scaleogram of Index %d for subject #%s, keyword of %s, on Channel %d, the lable is %s."%(i,sub,word,Channel,label))

#     scg_start = timeit.default_timer()
    ax = scg.cws(time, low_data, scales, yaxis = 'frequency', yscale='log')
    filename = f"sub_{sub}_idx_{i}_word_{word}_label_{label}_ch_{Channel}.png"
    filepath = os.path.join(output_dir, filename)
    plt.savefig(filepath, bbox_inches='tight',  pad_inches=0) 
    plt.close() 
#     scg_stop = timeit.default_timer()
#     scg_run_time = scg_stop - scg_start
#     print("Channel %d generate scalogram run time is "%(Channel), scg_run_time )
stop = timeit.default_timer()
run_time = stop - start
print(run_time)

################### achrive image and save as an array

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

image_paths = sorted(glob.glob(os.path.join(output_dir, "*.png")))
images = [transform(Image.open(p).convert("RGB")) for p in image_paths]
inputs_tensor = torch.stack(images).view(-1, 7, 3, 224, 224).to(device)

stop = timeit.default_timer()
run_time = stop - start
print(run_time)


################### test the data
sub_res = np.argmax(model(inputs_tensor)[0].cpu()[0].detach().numpy())
comd_res = np.argmax(model(inputs_tensor)[1].cpu()[0].detach().numpy())

print('Result: subject is #%d, and command is %s'%(sub_res+1,command_dict[comd_res]))
stop = timeit.default_timer()
run_time = stop - start
print(run_time)


0.2000052350340411
4.0709241579752415
4.099581883987412
Result: subject is #11, and command is Continue
4.143551651970483


In [ ]:
# if downsampling to about 375 data point per segment and change scale from 5-500 to 1-100

s = 5 # s is the subject No., here from 0 to 10, set a No. to get this subject data, e.g., set s=0 to get subject 1 data
sub = str(s+1).zfill(2) # get the subject #
word = 'Continue' # word is the keyword, select one from the list of keywords

if os.path.exists('inference_time_images'):
    shutil.rmtree('inference_time_images')
output_dir = "inference_time_images/test"
os.makedirs(output_dir, exist_ok=True)
start = timeit.default_timer()

#################################### convet the original mat file to scalogram image ####################################

################### get signal data from mat file 

#get the mat file of this word from the subject s and get the labels of all segmentations
mat_path = 'raw_data/subject_'+sub+'/'+word+'.mat'
mat = scipy.io.loadmat(mat_path)
labels = new_Alignment[(new_Alignment ["keyword"] == word)&(new_Alignment ["subject"] == s+1)]

signal_list = []
for Channel in range(1,8): # there are 7 channels for sEMG data in this dataset, from 1,2,...7
    c =  Channel - 1

    # get the data of this channel in this word from the subject s 
    if len(mat['tickrate'][0])>1:# just in case some files have a pause during recording
        for i in range(len(mat['tickrate'][0])): 
            s_data = mat['data'][0][int(mat['datastart'][c][i]):int(mat['dataend'][c][i])]
            s_data = np.transpose(s_data)
            data = np.append(data,s_data) if i > 0 else s_data
    else:
        data = mat['data'][0][int(mat['datastart'][c]):int(mat['dataend'][c])]
        data = np.transpose(data)  


    i = 15 # the index of the segmentation, i in range(len(labels)), in this dataset, i is from 0 to 50+
    label = labels["Label"].values[i]  # to get the lable of the segment i, Train, Val, or Test

    idnum = labels["wav_index"].values[i]-1  # the id of the output figure. 

    signal = data[labels["Start"].values[i]:labels["End"].values[i]] # get the original signal data based on the start and end index
    signal_list.append(signal)

stop = timeit.default_timer()
run_time = stop - start
print(run_time)
# signal_list
# 

################### convert signal data to Scalogram
scales = np.arange(1, 100) 
for Channel in range(1,8): # there are 7 channels for sEMG data in this dataset, from 1,2,...7
    c =  Channel - 1
    signal = signal_list[c]
    
    
#     downfreq_start = timeit.default_timer()
    low_data = []
    for j in range(len(signal)//160):
        low_data.append(signal[j*160:(j+1)*160].mean())
    low_data = np.array(low_data)

#     downfreq_stop = timeit.default_timer()
#     downfreq_run_time = downfreq_stop - downfreq_start
#     print("Channel %d downfreq run time is "%(Channel), downfreq_run_time )

    time = np.linspace(0, 1.5, num=len(low_data)) # 1.5 second per segment
#     print("sEMG Scaleogram of Index %d for subject #%s, keyword of %s, on Channel %d, the lable is %s."%(i,sub,word,Channel,label))

#     scg_start = timeit.default_timer()
    ax = scg.cws(time, low_data, scales, yaxis = 'frequency', yscale='log')
    filename = f"sub_{sub}_idx_{i}_word_{word}_label_{label}_ch_{Channel}.png"
    filepath = os.path.join(output_dir, filename)
    plt.savefig(filepath, bbox_inches='tight',  pad_inches=0) 
    plt.close() 
#     scg_stop = timeit.default_timer()
#     scg_run_time = scg_stop - scg_start
#     print("Channel %d generate scalogram run time is "%(Channel), scg_run_time )
stop = timeit.default_timer()
run_time = stop - start
print(run_time)

################### achrive image and save as an array

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

image_paths = sorted(glob.glob(os.path.join(output_dir, "*.png")))
images = [transform(Image.open(p).convert("RGB")) for p in image_paths]
inputs_tensor = torch.stack(images).view(-1, 7, 3, 224, 224).to(device)

stop = timeit.default_timer()
run_time = stop - start
print(run_time)


################### test the data
sub_res = np.argmax(model(inputs_tensor)[0].cpu()[0].detach().numpy())
comd_res = np.argmax(model(inputs_tensor)[1].cpu()[0].detach().numpy())

print('Result: subject is #%d, and command is %s'%(sub_res+1,command_dict[comd_res]))
stop = timeit.default_timer()
run_time = stop - start
print(run_time)


0.18454991094768047
0.9938006609445438
1.0193120120093226
Result: subject is #6, and command is Continue
1.0659265730064362


In [ ]:
# if downsampling to about 250 data point per segment and change scale from 5-500 to 1-100

s = 5 # s is the subject No., here from 0 to 10, set a No. to get this subject data, e.g., set s=0 to get subject 1 data
sub = str(s+1).zfill(2) # get the subject #
word = 'Continue' # word is the keyword, select one from the list of keywords

if os.path.exists('inference_time_images'):
    shutil.rmtree('inference_time_images')
output_dir = "inference_time_images/test"
os.makedirs(output_dir, exist_ok=True)
start = timeit.default_timer()

#################################### convet the original mat file to scalogram image ####################################

################### get signal data from mat file 

#get the mat file of this word from the subject s and get the labels of all segmentations
mat_path = 'raw_data/subject_'+sub+'/'+word+'.mat'
mat = scipy.io.loadmat(mat_path)
labels = new_Alignment[(new_Alignment ["keyword"] == word)&(new_Alignment ["subject"] == s+1)]

signal_list = []
for Channel in range(1,8): # there are 7 channels for sEMG data in this dataset, from 1,2,...7
    c =  Channel - 1

    # get the data of this channel in this word from the subject s 
    if len(mat['tickrate'][0])>1:# just in case some files have a pause during recording
        for i in range(len(mat['tickrate'][0])): 
            s_data = mat['data'][0][int(mat['datastart'][c][i]):int(mat['dataend'][c][i])]
            s_data = np.transpose(s_data)
            data = np.append(data,s_data) if i > 0 else s_data
    else:
        data = mat['data'][0][int(mat['datastart'][c]):int(mat['dataend'][c])]
        data = np.transpose(data)  


    i = 15 # the index of the segmentation, i in range(len(labels)), in this dataset, i is from 0 to 50+
    label = labels["Label"].values[i]  # to get the lable of the segment i, Train, Val, or Test

    idnum = labels["wav_index"].values[i]-1  # the id of the output figure. 

    signal = data[labels["Start"].values[i]:labels["End"].values[i]] # get the original signal data based on the start and end index
    signal_list.append(signal)

stop = timeit.default_timer()
run_time = stop - start
print(run_time)
# signal_list
# 

################### convert signal data to Scalogram
scales = np.arange(1, 100) 
for Channel in range(1,8): # there are 7 channels for sEMG data in this dataset, from 1,2,...7
    c =  Channel - 1
    signal = signal_list[c]
    
    
#     downfreq_start = timeit.default_timer()
    low_data = []
    for j in range(len(signal)//240):
        low_data.append(signal[j*240:(j+1)*240].mean())
    low_data = np.array(low_data)

#     downfreq_stop = timeit.default_timer()
#     downfreq_run_time = downfreq_stop - downfreq_start
#     print("Channel %d downfreq run time is "%(Channel), downfreq_run_time )

    time = np.linspace(0, 1.5, num=len(low_data)) # 1.5 second per segment
#     print("sEMG Scaleogram of Index %d for subject #%s, keyword of %s, on Channel %d, the lable is %s."%(i,sub,word,Channel,label))

#     scg_start = timeit.default_timer()
    ax = scg.cws(time, low_data, scales, yaxis = 'frequency', yscale='log')
    filename = f"sub_{sub}_idx_{i}_word_{word}_label_{label}_ch_{Channel}.png"
    filepath = os.path.join(output_dir, filename)
    plt.savefig(filepath, bbox_inches='tight',  pad_inches=0) 
    plt.close() 
#     scg_stop = timeit.default_timer()
#     scg_run_time = scg_stop - scg_start
#     print("Channel %d generate scalogram run time is "%(Channel), scg_run_time )
stop = timeit.default_timer()
run_time = stop - start
print(run_time)

################### achrive image and save as an array

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

image_paths = sorted(glob.glob(os.path.join(output_dir, "*.png")))
images = [transform(Image.open(p).convert("RGB")) for p in image_paths]
inputs_tensor = torch.stack(images).view(-1, 7, 3, 224, 224).to(device)

stop = timeit.default_timer()
run_time = stop - start
print(run_time)


################### test the data
sub_res = np.argmax(model(inputs_tensor)[0].cpu()[0].detach().numpy())
comd_res = np.argmax(model(inputs_tensor)[1].cpu()[0].detach().numpy())

print('Result: subject is #%d, and command is %s'%(sub_res+1,command_dict[comd_res]))
stop = timeit.default_timer()
run_time = stop - start
print(run_time)


0.20089086203370243
0.9109507469693199
0.9377956340322271
Result: subject is #6, and command is Continue
0.989383217995055


In [38]:
low_data.shape

(250,)

# NinaPro

In [15]:
movement_dict = {'E1_move_1': 0,
 'E1_move_10': 1,
 'E1_move_11': 2,
 'E1_move_12': 3,
 'E1_move_2': 4,
 'E1_move_3': 5,
 'E1_move_4': 6,
 'E1_move_5': 7,
 'E1_move_6': 8,
 'E1_move_7': 9,
 'E1_move_8': 10,
 'E1_move_9': 11,
 'E2_move_1': 12,
 'E2_move_10': 13,
 'E2_move_11': 14,
 'E2_move_12': 15,
 'E2_move_13': 16,
 'E2_move_14': 17,
 'E2_move_15': 18,
 'E2_move_16': 19,
 'E2_move_17': 20,
 'E2_move_2': 21,
 'E2_move_3': 22,
 'E2_move_4': 23,
 'E2_move_5': 24,
 'E2_move_6': 25,
 'E2_move_7': 26,
 'E2_move_8': 27,
 'E2_move_9': 28,
 'E3_move_1': 29,
 'E3_move_10': 30,
 'E3_move_11': 31,
 'E3_move_12': 32,
 'E3_move_13': 33,
 'E3_move_14': 34,
 'E3_move_15': 35,
 'E3_move_16': 36,
 'E3_move_17': 37,
 'E3_move_18': 38,
 'E3_move_19': 39,
 'E3_move_2': 40,
 'E3_move_20': 41,
 'E3_move_21': 42,
 'E3_move_22': 43,
 'E3_move_23': 44,
 'E3_move_3': 45,
 'E3_move_4': 46,
 'E3_move_5': 47,
 'E3_move_6': 48,
 'E3_move_7': 49,
 'E3_move_8': 50,
 'E3_move_9': 51}
movement_dict = {v: k for k, v in movement_dict.items()}
# movement_dict

In [16]:
###################### model 2 (keyword)###################### 

class keyword_net(nn.Module):
    def __init__(self):
        super(keyword_net, self).__init__()
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=1)            
        self.cvd = nn.Conv2d(in_channels=10, out_channels=3, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
        self.fc1_1 = nn.Linear(71910,128) 
        self.fc1_2 = nn.Linear(128,256) 
        self.keyword_output_1 = nn.Linear(256,27) 
        self.fc2_1 = nn.Linear(71910,512) # Dense_2_1 = tf.keras.layers.Dense(shape_2_1, activation=actv_fun_2_1,name='fc2_1')
        self.fc2_2 = nn.Linear(512,512) # tf.keras.layers.Dense(shape_2_2, activation=actv_fun_2_2,name='fc2_2')
        self.getalphafromy2 = nn.Linear(512,512)
        self.fc2_3 = nn.Linear(1024,512)
        self.keyword_output_2 = nn.Linear(512,52) # tf.keras.layers.Dense(10, activation='softmax',name='command_output')


    def forward(self, x):   
        x =  x.reshape(x.size(0), x.size(1), x.size(2), -1) 
        x = self.cvd(x)
        x = x.view(x.size(0), -1) 
        
        y1 = self.fc1_1(x)
        y1 = self.relu(y1)
        y1 = self.fc1_2(y1)
        y1 = self.sigmoid(y1) 
        y1 = self.keyword_output_1(y1)
        y1 = self.softmax(y1)
        
        y2 = self.fc2_1(x)
        y2 = self.relu(y2)
        y2 = self.fc2_2(y2)
        y2 = self.sigmoid(y2) 
        y2 = self.keyword_output_2(y2)
        y2 = self.softmax(y2)
        return y1,y2

In [17]:
class CombinedModel(nn.Module):
    def __init__(self, PRETRAIN_MODEL,keyword_net):
        super(CombinedModel, self).__init__()
        self.pretrain_model = PRETRAIN_MODEL
        self.keyword_net = keyword_net
        
    def forward(self, x):
#         print(x.shape)
#         print(x[:][0].shape)
        x1 = self.pretrain_model(x[:,0,:,:])
        x1 = torch.unsqueeze(x1, dim=1)
#         print(x1.shape)
        x2 = self.pretrain_model(x[:,1,:,:])
        x2 = torch.unsqueeze(x2, dim=1)
        x3 = self.pretrain_model(x[:,2,:,:])
        x3 = torch.unsqueeze(x3, dim=1)
        x4 = self.pretrain_model(x[:,3,:,:])
        x4 = torch.unsqueeze(x4, dim=1)        
        x5 = self.pretrain_model(x[:,4,:,:])
        x5 = torch.unsqueeze(x5, dim=1)        
        x6 = self.pretrain_model(x[:,5,:,:])
        x6 = torch.unsqueeze(x6, dim=1)        
        x7 = self.pretrain_model(x[:,6,:,:]) 
        x7 = torch.unsqueeze(x7, dim=1)    
        x8 = self.pretrain_model(x[:,7,:,:])
        x8 = torch.unsqueeze(x8, dim=1)        
        x9 = self.pretrain_model(x[:,8,:,:])
        x9 = torch.unsqueeze(x9, dim=1)        
        x10 = self.pretrain_model(x[:,9,:,:]) 
        x10 = torch.unsqueeze(x10, dim=1)    
                
        x = torch.cat((x1, x2, x3, x4, x5, x6, x7,x8,x9,x10), axis=1)
#         print(x.shape)
        y1, y2= self.keyword_net(x)
        return y1, y2
        

In [ ]:
PATH = "Final_model/NinaProDB1_group_Featurefuse_unfreeze_0412"
resnet = models.resnet18(pretrained=True)
PRETRAIN_MODEL = torch.nn.Sequential(*(list(resnet.children())[:-2])) 
model = CombinedModel(PRETRAIN_MODEL,keyword_net())
model.load_state_dict(torch.load(PATH))
model.to(device)
summary(model, ( 10,3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [ ]:
s = 17 # s is the subject No., here from 0 to 10, set a No. to get this subject data, e.g., set s=0 to get subject 1 data
sub = s+1 # get the subject #
print("subject: #"+str(sub)+"------------------------------------")

exc = 'E3'  #["E1","E2","E3"]

if os.path.exists('inference_time_images'):
    shutil.rmtree('inference_time_images')
output_dir = "inference_time_images/test"
os.makedirs(output_dir, exist_ok=True)
start = timeit.default_timer()

#################################### convet the original mat file to scalogram image ####################################

################### get signal data from mat file 

#get the mat file of this word from the subject s and get the labels of all segmentations
mat_path = 'raw_db1_data/S' + str(sub) + '_A1_' + exc + '.mat'
mat = scipy.io.loadmat(mat_path)

rept = 3
# based on the paper :Approximately one third of the movement repetitions were used to create the test set (repetition 2, 5 and 7 in database 1
if rept in [2,5,7]:
    label = 'Test'
else:
    label = 'Train'  

movemt = 1 #in range(1,mat['stimulus'].max()+1): # min is 0, means rest, so we select from 1
movement = exc + '_move_' + str(movemt)
print(movement)
mask = (mat['stimulus'] == movemt) & (mat['repetition'] == rept)

signal_list = []
for chan in range(0,10):
    data = mat['emg'][np.transpose(mask)[0]][:,chan]
    signal_list.append(data)

stop = timeit.default_timer()
run_time = stop - start
print(run_time)
# signal_list
# 

################### convert signal data to Scalogram
for Channel in range(1,11): # there are 7 channels for sEMG data in this dataset, from 1,2,...7
    c =  Channel - 1
    signal = signal_list[c]
    time = np.arange(1, signal.shape[0]+1)
    ax = scg.cws(time, signal, np.arange(1, 100), wavelet='cmor0.3-0.3', yscale='log', yaxis = 'frequency' )
    filename = f"sub_{sub}_rept_{rept}_movement_{movement}_label_{label}_ch_{Channel}.png"
    filepath = os.path.join(output_dir, filename)
    plt.savefig(filepath, bbox_inches='tight',  pad_inches=0) 
    plt.close() 
    
################### achrive image and save as an array

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

image_paths = sorted(glob.glob(os.path.join(output_dir, "*.png")))
images = [transform(Image.open(p).convert("RGB")) for p in image_paths]
inputs_tensor = torch.stack(images).view(-1, 10, 3, 224, 224).to(device)

stop = timeit.default_timer()
run_time = stop - start
print(run_time)


################### test the data
sub_res = np.argmax(model(inputs_tensor)[0].cpu()[0].detach().numpy())
comd_res = np.argmax(model(inputs_tensor)[1].cpu()[0].detach().numpy())

print('Result: subject is #%d, and movement is %s'%(sub_res+1,movement_dict[comd_res]))
stop = timeit.default_timer()
run_time = stop - start
print(run_time)

subject: #18------------------------------------
E3_move_1
0.1736458750674501
1.694076725980267
Result: subject is #14, and movement is E2_move_1
1.7684078000020236


In [20]:
comd_res

12